In [1]:
from elasticsearch import Elasticsearch
import pandas as pd
from elasticsearch.helpers import bulk
from elasticsearch import helpers
import numpy as np

In [2]:
es = Elasticsearch()

#name of the created index
index_name = "tweets"

In [3]:
# index settings
mapping = {
    "settings": {
        "number_of_shards": "5",
        "number_of_replicas": "1"
    },
    
    "mappings": {
        "properties": {
            "created_at_m": { 'type': 'double'},
            "created_at_y": {'type': 'double'},
            "geo": {"type": "geo_point"},
            "text": {'type': 'text'},
        },
    }
}
# create index
if es.indices.exists(index_name):
    es.indices.delete(index=index_name, ignore = 400)
es.indices.create(index=index_name, body=mapping)

D:\Anaconda\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'tweets'}

In [4]:
tweets_file = pd.read_json('boulder_flood_geolocated_tweets.json',lines=True)

In [5]:
tweets_file_df = pd.DataFrame(tweets_file)

In [6]:
tweets_file_df

,created_at,id,id_str,text,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,favorite_count,favorited,retweeted,possibly_sensitive,lang,extended_entities,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status
0,2013-12-31 07:14:22+00:00,417916626596806656,417916626596806656,Boom bitch get out the way! #drunk #islands #g...,False,"{'hashtags': [{'text': 'drunk', 'indices': [28...","<a href=""http://instagram.com"" rel=""nofollow"">...",NaN,NaN,NaN,...,0,False,False,0.0,en,NaN,NaN,NaN,NaN,NaN
1,2013-12-31 18:49:31+00:00,418091565161017345,418091565161017344,@WeatherDude17 Not that revved up yet due to m...,False,"{'hashtags': [{'text': 'snow', 'indices': [108...","<a href=""https://about.twitter.com/products/tw...",4.180914e+17,4.180914e+17,1.214464e+09,...,0,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN
2,2013-12-30 20:29:20+00:00,417754295455723520,417754295455723520,Story of my life! 😂 #boulder http://t.co/ZMfNK...,False,"{'hashtags': [{'text': 'boulder', 'indices': [...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,...,0,False,False,0.0,en,"{'media': [{'id': 417754295334088704, 'id_str'...",NaN,NaN,NaN,NaN
3,2013-12-30 23:02:29+00:00,417792838428925952,417792838428925952,We're looking for the two who came to help a c...,False,"{'hashtags': [{'text': 'Boulder', 'indices': [...","<a href=""http://www.tweetcaster.com"" rel=""nofo...",NaN,NaN,NaN,...,0,False,False,0.0,en,NaN,NaN,NaN,NaN,NaN
4,2014-01-01 06:12:15+00:00,418263379027820544,418263379027820544,Happy New Year #Boulder !!!! What are some of ...,False,"{'hashtags': [{'text': 'Boulder', 'indices': [...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,...,0,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18816,2013-09-12 04:12:20+00:00,378008148759420928,378008148759420928,RT @EricRobertson: A more representative flood...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://www.twitter.com"" rel=""nofollow...",NaN,NaN,NaN,...,0,False,False,0.0,en,NaN,{'created_at': 'Thu Sep 12 03:54:00 +0000 2013...,NaN,NaN,NaN
18817,2013-09-12 04:03:47+00:00,378005998872432640,378005998872432640,Warning. Flash flood of boulder creek is immin...,False,"{'hashtags': [{'text': 'boulderflood', 'indice...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,...,0,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN
18818,2013-09-12 04:11:56+00:00,378008050335875074,378008050335875072,I just got an “Emergency Alert”! Nice to know ...,False,"{'hashtags': [{'text': 'cowx', 'indices': [112...","<a href=""http://tapbots.com/tweetbot"" rel=""nof...",NaN,NaN,NaN,...,0,False,False,0.0,en,"{'media': [{'id': 378008050017136640, 'id_str'...",NaN,NaN,NaN,NaN
18819,2013-09-12 04:05:59+00:00,378006549735538688,378006549735538688,RT @dailycamera: RT @CUBoulderPolice: Some fl...,False,"{'hashtags': [{'text': 'CUBoulder', 'indices':...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,...,0,False,False,NaN,en,NaN,{'created_at': 'Thu Sep 12 04:05:17 +0000 2013...,NaN,NaN,NaN


In [7]:
tweets_file_df.columns

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'possibly_sensitive', 'lang',
       'extended_entities', 'retweeted_status', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status'],
      dtype='object')

In [8]:
TF = []

In [9]:
df = tweets_file_df[tweets_file_df['geo'].notna()]

In [10]:
df.reset_index(inplace=True)

In [11]:
df['geo']

0       {'type': 'Point', 'coordinates': [34.14628356,...
1       {'type': 'Point', 'coordinates': [52.22500698,...
2       {'type': 'Point', 'coordinates': [-37.80312131...
3       {'type': 'Point', 'coordinates': [40.01544423,...
4       {'type': 'Point', 'coordinates': [40.01579474,...
                              ...                        
3938    {'type': 'Point', 'coordinates': [40.00217227,...
3939    {'type': 'Point', 'coordinates': [39.70087895,...
3940    {'type': 'Point', 'coordinates': [39.77646518,...
3941    {'type': 'Point', 'coordinates': [40.03908039,...
3942    {'type': 'Point', 'coordinates': [39.65430591,...
Name: geo, Length: 3943, dtype: object

In [12]:
for i in range(len(df)):
    TF.append([df['geo'][i]['coordinates']])

In [13]:
TF=pd.DataFrame(TF)

In [14]:
TF.columns =['geo']

In [15]:
TF['created_at'] = df['created_at']

In [16]:
TF['text'] = df['text']

In [17]:
TF

,geo,created_at,text
0,"[34.14628356, -118.10041174]",2013-12-31 07:14:22+00:00,Boom bitch get out the way! #drunk #islands #g...
1,"[52.22500698, 0.13429814]",2013-12-30 20:29:20+00:00,Story of my life! 😂 #boulder http://t.co/ZMfNK...
2,"[-37.80312131, 144.98467167]",2014-01-01 06:12:15+00:00,Happy New Year #Boulder !!!! What are some of ...
3,"[40.01544423, -105.27725101]",2013-12-31 22:48:10+00:00,Last sunset here in #boulder #flatirons @ Dush...
4,"[40.01579474, -105.2769568]",2014-01-01 04:57:01+00:00,Ringing in the #NewYear @BMoCA for their NYE a...
...,...,...,...
3938,"[40.00217227, -105.0837946]",2013-09-12 05:08:40+00:00,“@dailycamera: RT @JoeCarmenRubino: #Boulder C...
3939,"[39.70087895, -104.95341443]",2013-09-12 05:09:24+00:00,Wow. “@jrbates: Update: @dailycamera reports s...
3940,"[39.77646518, -104.87204539]",2013-09-12 04:36:22+00:00,Dealing with heavy rain along I-70E near Auror...
3941,"[40.03908039, -105.27772709]",2013-09-12 04:35:37+00:00,CU kids having fun in the flooding underpass. ...


In [18]:
df = pd.DatetimeIndex(TF['created_at'])

TF['created_at_m'] = df.month
TF['created_at_y'] = df.year
TF.drop(columns='created_at', inplace=True)

In [19]:
TF

,geo,text,created_at_m,created_at_y
0,"[34.14628356, -118.10041174]",Boom bitch get out the way! #drunk #islands #g...,12,2013
1,"[52.22500698, 0.13429814]",Story of my life! 😂 #boulder http://t.co/ZMfNK...,12,2013
2,"[-37.80312131, 144.98467167]",Happy New Year #Boulder !!!! What are some of ...,1,2014
3,"[40.01544423, -105.27725101]",Last sunset here in #boulder #flatirons @ Dush...,12,2013
4,"[40.01579474, -105.2769568]",Ringing in the #NewYear @BMoCA for their NYE a...,1,2014
...,...,...,...,...
3938,"[40.00217227, -105.0837946]",“@dailycamera: RT @JoeCarmenRubino: #Boulder C...,9,2013
3939,"[39.70087895, -104.95341443]",Wow. “@jrbates: Update: @dailycamera reports s...,9,2013
3940,"[39.77646518, -104.87204539]",Dealing with heavy rain along I-70E near Auror...,9,2013
3941,"[40.03908039, -105.27772709]",CU kids having fun in the flooding underpass. ...,9,2013


In [20]:
tweets = TF.values.tolist()


In [21]:
tweets[1000]

[[40.01341063, -105.27946343],
 'Planning a bike trip soon? Johnny Jets travel product of the week is designed by #Boulder native &amp; sold by Nite Ize  http://t.co/lviqR09P9v',
 11,
 2013]

In [22]:
tweets_list = [
    {
        '_index': index_name,
        '_type': '_doc',
        '_source':{
            "location": tweet[0],
            "created_at_m": tweet[2],
            "created_at_y": tweet[3],            
            "content": tweet[1],
        }
    }
    for tweet in tweets
]

In [23]:
# index settings
mapping = {
    "settings": {
        "number_of_shards": "10",
        "number_of_replicas": "1"
    },
    
    "mappings": {
        "properties": {
            "created_at_m": { 'type': 'double'},
            "created_at_y": {'type': 'double'},
            "geo": {"type": "geo_point"},
            "text": {'type': 'text'},
        },
    }
}
# create index
if es.indices.exists(index_name):
    es.indices.delete(index=index_name, ignore = 400)
es.indices.create(index=index_name, body=mapping)

D:\Anaconda\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'tweets'}

In [24]:
helpers.bulk(es,tweets_list)

D:\Anaconda\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


(3943, [])

In [25]:
Query = {'query': 
         {'bool': {'filter':
                   {'match':
                     {'content': 'hello'}}}}}

In [26]:
res = es.search(index="tweets", body=Query,  size=10000)
print(res)


{'took': 19, 'timed_out': False, '_shards': {'total': 10, 'successful': 10, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 6, 'relation': 'eq'}, 'max_score': 0.0, 'hits': [{'_index': 'tweets', '_type': '_doc', '_id': 'Br6l6H0BH17YXLOaJdbN', '_score': 0.0, '_source': {'location': [39.97210419, -105.24599685], 'created_at_m': 12, 'created_at_y': 2013, 'content': 'Hello Winter. -1 degrees and gorgeous! #winter #boulder #colorado #flatirons @ Fairview High School http://t.co/zsIJR4ChKH'}}, {'_index': 'tweets', '_type': '_doc', '_id': 'FL6l6H0BH17YXLOaJ9iu', '_score': 0.0, '_source': {'location': [40.04802613, -105.28807801], 'created_at_m': 11, 'created_at_y': 2013, 'content': 'Hello Boulder.  #boulder #happyveteransday 🇺🇸 @ Wonderland Lake Park http://t.co/mujYSFwRVO'}}, {'_index': 'tweets', '_type': '_doc', '_id': 'cr6l6H0BH17YXLOaJ9mv', '_score': 0.0, '_source': {'location': [40.01918011, -105.27730954], 'created_at_m': 10, 'created_at_y': 2013, 'content': 'Hello Colorado ! \nS

In [27]:
def topleftquery(lon1, lat1, lon2, lat2):
    return es.search(index="tweets", body=
        {
            "query": {
                "bool":{
                    "filter":
                        [{"geo_bounding_box": {"coordinates": {"bottom_right": [long2, long2], "top_left": [lat1, long1]}}}]
                }
            }
        }
    )

In [28]:
lat1=54.59984185377703
long1=-135.2381320408579
lat2= 32.80910207882173
long2=-86.5396755285213

res = topleftquery(lat1,long1,lat2,long2)

RequestError: RequestError(400, 'x_content_parse_exception', '[1:169] [bool] failed to parse field [filter]')